In [2]:
import re
import constants
import os
import pandas as pd
import multiprocessing
import time
from tqdm import tqdm
from pathlib import Path
from functools import partial
import urllib.request
from PIL import Image

def common_mistake(unit):
    if unit in constants.allowed_units:
        return unit
    if unit.replace('ter', 'tre') in constants.allowed_units:
        return unit.replace('ter', 'tre')
    if unit.replace('feet', 'foot') in constants.allowed_units:
        return unit.replace('feet', 'foot')
    return unit

def parse_string(s):
    s_stripped = "" if s is None or str(s) == 'nan' else s.strip()
    if s_stripped == "":
        return None, None
    pattern = re.compile(r'^-?\d+(\.\d+)?\s+[a-zA-Z\s]+$')
    if not pattern.match(s_stripped):
        raise ValueError(f"Invalid format in {s}")
    parts = s_stripped.split(maxsplit=1)
    number = float(parts[0])
    unit = common_mistake(parts[1])
    if unit not in constants.allowed_units:
        raise ValueError(f"Invalid unit [{unit}] found in {s}. Allowed units: {constants.allowed_units}")
    return number, unit

def create_placeholder_image(image_save_path):
    try:
        placeholder_image = Image.new('RGB', (100, 100), color='black')
        placeholder_image.save(image_save_path)
    except Exception as e:
        print(f"Error creating placeholder image: {e}")

def download_image(image_link, save_folder, retries=3, delay=3):
    if not isinstance(image_link, str) or not image_link.strip():
        print(f"Skipping invalid image link: {image_link}")
        return

    try:
        filename = Path(image_link).name
        image_save_path = os.path.join(save_folder, filename)

        if os.path.exists(image_save_path):
            print(f"Image already exists: {image_save_path}")
            return

        for _ in range(retries):
            try:
                urllib.request.urlretrieve(image_link, image_save_path)
                return
            except Exception as e:
                print(f"Error downloading {image_link}: {e}")
                time.sleep(delay)

        # Create a placeholder image if all download attempts fail
        create_placeholder_image(image_save_path)
    except Exception as e:
        print(f"Unhandled error with link {image_link}: {e}")

def download_images(image_links, download_folder, allow_multiprocessing=True):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    if allow_multiprocessing:
        download_image_partial = partial(
            download_image, save_folder=download_folder, retries=3, delay=3)

        # Set the number of workers to 60 or lower to avoid the issue on Windows
        num_workers = min(60, multiprocessing.cpu_count())

        with multiprocessing.Pool(num_workers) as pool:
            list(tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)))
            pool.close()
            pool.join()
    else:
        for image_link in tqdm(image_links, total=len(image_links)):
            download_image(image_link, save_folder=download_folder, retries=3, delay=3)

# Step 1: Read cleaned data from CSV
cleaned_csv_path = r"D:\Hackathon\66e31d6ee96cd_student_resource_33\student_resource 3\dataset\Weightt_cleaned.csv"

# Load the CSV file into a DataFrame, ensuring low_memory is set to False to handle mixed data types properly
df = pd.read_csv(cleaned_csv_path, low_memory=False)

# Step 2: Extract image links from the 'img_link' column
# Convert all values to strings, drop NaNs, and strip whitespace to clean the links
image_links = df['image_link'].astype(str).dropna().map(str.strip).tolist()

# Step 3: Set the folder where images will be downloaded
download_folder = r"D:\Hackathon\66e31d6ee96cd_student_resource_33\student_resource 3\dataset\images"

# Step 4: Download images using the provided function
download_images(image_links, download_folder, allow_multiprocessing=False)


  5%|▍         | 6197/131187 [20:06<6:26:32,  5.39it/s] 

Error downloading https://m.media-amazon.com/images/I/DzP2RMRQO0.jpg: HTTP Error 400: Bad Request
Error downloading https://m.media-amazon.com/images/I/DzP2RMRQO0.jpg: HTTP Error 400: Bad Request
Error downloading https://m.media-amazon.com/images/I/DzP2RMRQO0.jpg: HTTP Error 400: Bad Request


 14%|█▍        | 19021/131187 [1:10:35<6:56:14,  4.49it/s] 


KeyboardInterrupt: 

In [1]:
import re
import constants
import os
import pandas as pd
import multiprocessing
import time
from tqdm import tqdm
from pathlib import Path
from functools import partial
import urllib.request
from PIL import Image

def common_mistake(unit):
    if unit in constants.allowed_units:
        return unit
    if unit.replace('ter', 'tre') in constants.allowed_units:
        return unit.replace('ter', 'tre')
    if unit.replace('feet', 'foot') in constants.allowed_units:
        return unit.replace('feet', 'foot')
    return unit

def parse_string(s):
    s_stripped = "" if s is None or str(s) == 'nan' else s.strip()
    if s_stripped == "":
        return None, None
    pattern = re.compile(r'^-?\d+(\.\d+)?\s+[a-zA-Z\s]+$')
    if not pattern.match(s_stripped):
        raise ValueError(f"Invalid format in {s}")
    parts = s_stripped.split(maxsplit=1)
    number = float(parts[0])
    unit = common_mistake(parts[1])
    if unit not in constants.allowed_units:
        raise ValueError(f"Invalid unit [{unit}] found in {s}. Allowed units: {constants.allowed_units}")
    return number, unit

def create_placeholder_image(image_save_path):
    try:
        placeholder_image = Image.new('RGB', (100, 100), color='black')
        placeholder_image.save(image_save_path)
        print(f"Placeholder image created at {image_save_path}")
    except Exception as e:
        print(f"Error creating placeholder image: {e}")

def download_image(image_link, save_folder, retries=3, delay=3):
    if not isinstance(image_link, str) or not image_link.strip():
        print(f"Skipping invalid image link: {image_link}")
        return

    try:
        filename = Path(image_link).name
        image_save_path = os.path.join(save_folder, filename)

        if os.path.exists(image_save_path):
            print(f"Image already exists: {image_save_path}")
            return

        print(f"Attempting to download image from {image_link}")

        for attempt in range(retries):
            try:
                urllib.request.urlretrieve(image_link, image_save_path)
                print(f"Downloaded {image_link} to {image_save_path}")
                return
            except Exception as e:
                print(f"Error on attempt {attempt+1} to download {image_link}: {e}")
                time.sleep(delay)

        # Create a placeholder image if all download attempts fail
        create_placeholder_image(image_save_path)
    except Exception as e:
        print(f"Unhandled error with link {image_link}: {e}")

def download_images(image_links, download_folder, allow_multiprocessing=True):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    if allow_multiprocessing:
        download_image_partial = partial(
            download_image, save_folder=download_folder, retries=3, delay=3)

        num_workers = min(60, multiprocessing.cpu_count())

        with multiprocessing.Pool(num_workers) as pool:
            list(tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)))
            pool.close()
            pool.join()
    else:
        for image_link in tqdm(image_links, total=len(image_links)):
            download_image(image_link, save_folder=download_folder, retries=3, delay=3)

# Step 1: Read cleaned data from CSV
cleaned_csv_path = r"D:\Hackathon\66e31d6ee96cd_student_resource_33\student_resource 3\dataset\Weightt_cleaned.csv"

# Load the CSV file into a DataFrame, ensuring low_memory is set to False to handle mixed data types properly
df = pd.read_csv(cleaned_csv_path, low_memory=False)

# Step 2: Extract image links from the 'image_link' column
# Convert all values to strings, drop NaNs, and strip whitespace to clean the links
image_links = df['image_link'].astype(str).dropna().map(str.strip).tolist()

# Limit to the first 30 image links
image_links = image_links[:30]

# Step 3: Set the folder where images will be downloaded
download_folder = r"D:\Hackathon\66e31d6ee96cd_student_resource_33\student_resource 3\dataset\images"

# Step 4: Download images using the provided function
download_images(image_links, download_folder, allow_multiprocessing=False)


  0%|          | 0/30 [00:00<?, ?it/s]

Attempting to download image from https://m.media-amazon.com/images/I/61I9XdN6OFL.jpg


  7%|▋         | 2/30 [00:00<00:08,  3.12it/s]

Downloaded https://m.media-amazon.com/images/I/61I9XdN6OFL.jpg to D:\Hackathon\66e31d6ee96cd_student_resource_33\student_resource 3\dataset\images\61I9XdN6OFL.jpg
Attempting to download image from https://m.media-amazon.com/images/I/61BZ4zrjZXL.jpg
Downloaded https://m.media-amazon.com/images/I/61BZ4zrjZXL.jpg to D:\Hackathon\66e31d6ee96cd_student_resource_33\student_resource 3\dataset\images\61BZ4zrjZXL.jpg
Attempting to download image from https://m.media-amazon.com/images/I/612mrlqiI4L.jpg


 13%|█▎        | 4/30 [00:00<00:05,  5.16it/s]

...

Downloaded https://m.media-amazon.com/images/I/61XGDKap+JL.jpg to D:\Hackathon\66e31d6ee96cd_student_resource_33\student_resource 3\dataset\images\61XGDKap+JL.jpg
Attempting to download image from https://m.media-amazon.com/images/I/715vVcWJxGL.jpg
Downloaded https://m.media-amazon.com/images/I/715vVcWJxGL.jpg to D:\Hackathon\66e31d6ee96cd_student_resource_33\student_resource 3\dataset\images\715vVcWJxGL.jpg
Attempting to download image from https://m.media-amazon.com/images/I/613v+2W4UwL.jpg


 93%|█████████▎| 28/30 [00:06<00:00,  5.60it/s]

Downloaded https://m.media-amazon.com/images/I/613v+2W4UwL.jpg to D:\Hackathon\66e31d6ee96cd_student_resource_33\student_resource 3\dataset\images\613v+2W4UwL.jpg
Attempting to download image from https://m.media-amazon.com/images/I/71+fn9TWQmL.jpg
Downloaded https://m.media-amazon.com/images/I/71+fn9TWQmL.jpg to D:\Hackathon\66e31d6ee96cd_student_resource_33\student_resource 3\dataset\images\71+fn9TWQmL.jpg
Attempting to download image from https://m.media-amazon.com/images/I/71aKgRRQ2wL.jpg


100%|██████████| 30/30 [00:06<00:00,  4.40it/s]

Downloaded https://m.media-amazon.com/images/I/71aKgRRQ2wL.jpg to D:\Hackathon\66e31d6ee96cd_student_resource_33\student_resource 3\dataset\images\71aKgRRQ2wL.jpg
Attempting to download image from https://m.media-amazon.com/images/I/71rKXZJrh4L.jpg
Downloaded https://m.media-amazon.com/images/I/71rKXZJrh4L.jpg to D:\Hackathon\66e31d6ee96cd_student_resource_33\student_resource 3\dataset\images\71rKXZJrh4L.jpg
